In [ ]:
import pyemma
pyemma.__version__
import random as rndm

In [ ]:
import matplotlib as mpltlib
import matplotlib.pylab as plt
import numpy as np
#import nglview as nv
%pylab inline
import mdtraj
import numpy as np

import pyemma.util.contexts


In [ ]:
import pyemma.coordinates as coor
import pyemma.msm as msm
import pyemma.plots as mplt

In [ ]:
indir = '.'
topfile =  'hdim-oct.inpcrd.pdb'
traj_list = []
for filename in os.listdir(indir):
    if filename.endswith('.nc'):
        traj_list.append(os.path.join(indir,filename))


In [ ]:
atom_mol = 145
num_mol = 2

topology = mdtraj.load(topfile).topology

In [ ]:
feat = coor.featurizer(topfile)
feat.add_backbone_torsions(deg=True)
inp = coor.load(traj_list, features=feat)

In [ ]:
traj_length = 200
inp = coor.source(traj_list, feat)
print ('number of trajectories = ',inp.number_of_trajectories())
print ('trajectory length = ',inp.trajectory_length(0))
print ('trajectory time step = ', traj_length/ (inp.trajectory_length(0)-1),'ns')
print ('number of dimension = ',inp.dimension())

In [ ]:
lag = 100
dim = 2
tica_obj = coor.tica(inp, lag = lag, dim = dim, kinetic_map = True)

In [ ]:
Y = tica_obj.get_output()

Y1 = np.concatenate(Y)

In [ ]:
figure(figsize=(9,7))
plt.subplot2grid((2,1),(0,0))
plt.plot(Y1[:,0])
plt.ylabel('IC 1')
plt.subplot2grid((2,1),(1,0))
plt.plot(Y1[:,1])
plt.ylabel('IC 2')

plt.xlabel('timesteps')
save_figure('icvstime.eps')

In [ ]:
mplt.plot_feature_histograms(Y1, feature_labels=['IC1','IC2'])

In [ ]:
IC1 = Y1.T[0]
IC2 = Y1.T[1]

fig, ax, misc = pyemma.plots.plot_density(IC1, IC2, logscale=True)
plt.ylabel('IC2')
plt.xlabel('IC1')

In [ ]:
tica0 = np.array([])
tica1 = np.array([])
for j in range(len(Y)):
    tica0 = np.concatenate((tica0, Y[j][:,0]))

for j in range(len(Y)):
    tica1 = np.concatenate((tica1, Y[j][:,1]))
    
# histogram data
z,x,y = np.histogram2d(tica0, tica1, bins=200)
extent = (x.min(), x.max(), y.min(), y.max()) # extent of the plot
# compute free energies
F = -np.log(z)
F[F == inf] = -1000
maxval = np.amax(F)
F[F == -1000] = maxval
plt.figure(figsize=(6,5))
plt.contourf(F.T, 50, cmap=plt.cm.afmhot, extent = extent)
plt.colorbar()
plt.ylabel('IC2')
plt.xlabel('IC1')
plt.show()

del(tica0)
del(tica1)

save_figure('ic1vsic2.eps')

In [ ]:
cluster_centers = 500
cl = coor.cluster_kmeans(data = Y, k= cluster_centers, max_iter=200)
# for later use we save the discrete trajectories and cluster center coordinates:
dtrajs = cl.dtrajs
cc_x = cl.clustercenters[:,0]
cc_y = cl.clustercenters[:,1]

In [ ]:
mplt.plot_free_energy(np.vstack(Y)[:,0], np.vstack(Y)[:,1]);
#plt.contourf(F.T, 50, cmap=plt.cm.hot, extent=extent)
plot(cc_x,cc_y, linewidth = 0.001, marker='.', c = 'C1')
save_figure('cluster.eps')

In [ ]:
lags = [1,2,5,10,15,20,25,30,50,75,100,125, 150,175, 200]
its = msm.its(dtrajs, lags=lags, n_jobs=1, nits=15)

In [ ]:
figure(figsize=(6,5))
matplotlib.rcParams.update({'font.size': 30})
mplt.plot_implied_timescales(its)
save_figure('timescales.png')
plt.xticks([0,50,100,150,200])
save_figure('timescales.eps')

In [ ]:
lag = 50
M = msm.estimate_markov_model(dtrajs, lag)

In [ ]:
print ('fraction of states used = ', M.active_state_fraction)
print ('fraction of counts used = ', M.active_count_fraction)

In [ ]:
plot(M.timescales(),linewidth=0,marker='o')
xlabel('index'); ylabel('timescale (10 ns)'); xlim(-0.5,25.5)

In [ ]:
plot(M.timescales()[:-1]/M.timescales()[1:], linewidth=0,marker='o')
xlabel('index'); ylabel('timescale separation'); xlim(-0.5, 25.5); ylim(1, 5);
save_figure('timescale_ratio.png')

In [ ]:
n_sets = 7
ck = M.cktest(n_sets, mlags=10, err_est=False, show_progress=True, n_jobs=1)

matplotlib.rcParams.update({'font.size': 20})
mplt.plot_cktest(ck, diag=True, figsize=(50,50), layout=(n_sets, n_sets), padding_top=0.1, y01=False, padding_between=0.3, dt=0.004, units='ns')
save_figure('cktest.png')

In [ ]:
# Coarse graining systems

M.pcca(n_sets)
pcca_dist = M.metastable_distributions
membership = M.metastable_memberships  # get PCCA memberships

In [ ]:
pcca_samples = M.sample_by_distributions(pcca_dist, 1000)

In [ ]:
outfiles = []
for k in range(0, n_sets):
    intstring = str(k+1)
    filename = 'samples_pcca'+ intstring + '.nc'
    outfiles.append(filename)
coor.save_trajs(inp, pcca_samples, outfiles = outfiles)

In [ ]:
figure(figsize=(20,20))
pcca_sets = M.metastable_sets
fig,ax = mplt.plot_free_energy(np.vstack(Y)[:,0], np.vstack(Y)[:,1], cmap = 'afmhot')
ax.set_xlabel('IC 1')
ax.set_ylabel('IC 2')
# ax.set_xlim(-2,2)
# ax.set_ylim(-3,3)
colornames = list(matplotlib.colors.cnames.keys())
#colornames
colorselect = rndm.sample(colornames, n_sets + 1)
for k in range(n_sets):
    scatter(cl.clustercenters[pcca_sets[k],0], cl.clustercenters[pcca_sets[k],1], color=colorselect[k], s=10)
    
fig.savefig('free_energy_with_cluster.png',dpi = 300)

In [ ]:
hmm = M.coarse_grain(n_sets)

In [ ]:
# np.set_printoptions(precision=12, suppress=True)
# hmm.stationary_distribution

In [ ]:
np.set_printoptions(precision=4, suppress=True)
print(hmm.transition_matrix)

In [ ]:
def avg_by_set(x, sets):
    # compute mean positions of sets. This is important because of some technical points the set order 
    # in the coarse-grained TPT object can be different from the input order.
    avg = np.zeros(len(sets))
    for i in range(len(sets)):
        I = list(sets[i])
        avg[i] = np.mean(x[I])
    return avg

In [ ]:
pcca_sets = hmm.metastable_sets
xavg = avg_by_set(cc_x, pcca_sets)
yavg = avg_by_set(cc_y, pcca_sets)
avgpos = np.zeros((n_sets,2))
avgpos[:,0] = xavg
avgpos[:,1] = yavg

In [ ]:
fig,_ = mplt.plot_markov_model(hmm, pos=avgpos, minflux=1e-5, arrow_scale=0.8, arrow_label_format=' ', state_colors='skyblue', figpadding=0.25, max_width =20, max_height = 20)
gca().set_frame_on(False)

fig.savefig('network.eps',dpi = 300)

In [ ]:
A = [1]
B = [4]
flux = pyemma.msm.tpt(M1, A, B)

In [ ]:
cg, cgflux = flux.coarse_grain(hmm.metastable_sets)

In [ ]:
# Probable pathways

paths, path_fluxes = cgflux.pathways(fraction=1.00)
print('percentage       \tpath')
print('-------------------------------------')
for i in range(len(paths)):
    print(np.round(path_fluxes[i] / np.sum(path_fluxes), 3),' \t', paths[i])

In [ ]:
flux.pathways()

In [ ]:
(paths,pathfluxes) =flux.pathways()
cumflux = 0
print("Path flux\t\t%path\t%of total\tpath")
for i in range(len(paths)):
    cumflux += pathfluxes[i]
    print(pathfluxes[i],'\t','%3.1f'%(100.0*pathfluxes[i]/flux.total_flux),'%\t','%3.1f'%(100.0*cumflux/flux.total_flux),'%\t\t',paths[i])

In [ ]:
mplt.plot_flux(flux, pos=avgpos, flux_scale=100.0/flux.total_flux, arrow_label_format="%3.1f",state_colors='skyblue',state_scale=0.1)